In [1]:
import os

from datetime import timezone, datetime

from timescaledb_util import TimeScaleDBUtil
from trades_download_util import TradesDownloadUtil
import pandas as pd

pd.options.display.precision = 10

# PostgreSQL設定
pg_config = {
    'user': os.environ['POSTGRES_USER'],
    'password': os.environ['POSTGRES_PASSWORD'],
    'host': os.environ['POSTGRES_HOST'],
    'port': os.environ['POSTGRES_PORT'],
    'database': os.environ['POSTGRES_DATABASE']
}

dbutil = TimeScaleDBUtil(user = pg_config['user'], password = pg_config['password'], host = pg_config['host'], port = pg_config['port'], database = pg_config['database'])

In [2]:
table_name = dbutil.get_trade_table_name('bitfinex2', 'XRP/BTC')
df_trades_a = dbutil.read_sql_query(sql = f'SELECT * FROM "{table_name}" WHERE datetime < \'2021-12-26 00:00:00+00\' ORDER BY datetime')
df_trades_a = df_trades_a.rename(columns={'price': 'price_a'})

In [3]:
table_name = dbutil.get_trade_table_name('poloniex', 'XRP/BTC')
df_trades_b = dbutil.read_sql_query(sql = f'SELECT * FROM "{table_name}" WHERE datetime < \'2021-12-26 00:00:00+00\' ORDER BY datetime')
df_trades_b = df_trades_b.rename(columns={'price': 'price_b'})

In [4]:
pd.set_option('display.max_rows', 10)
pd.set_option("display.min_rows", 10)
df_trades_a

,datetime,id,side,price_a,amount
0,2021-01-01 00:00:16.586000+00:00,557066356,sell,0.00000758,833.51076895
1,2021-01-01 00:00:16.586000+00:00,557066355,sell,0.00000758,277.83692298
2,2021-01-01 00:01:19.722000+00:00,557066362,buy,0.00000759,2370.22951037
3,2021-01-01 00:01:19.730000+00:00,557066363,buy,0.00000759,4402.35659604
4,2021-01-01 00:03:28.863000+00:00,557066364,buy,0.00000760,3109.18009407
...,...,...,...,...,...
1635641,2021-12-25 23:48:22.390000+00:00,939587876,buy,0.00001834,50.00000000
1635642,2021-12-25 23:48:59.565000+00:00,939587883,sell,0.00001832,50.00000000
1635643,2021-12-25 23:57:46.384000+00:00,939588084,buy,0.00001836,2.16858086
1635644,2021-12-25 23:58:33.796000+00:00,939588153,buy,0.00001836,4.59380644


In [5]:
pd.set_option('display.max_rows', 10)
pd.set_option("display.min_rows", 10)
df_trades_b

,datetime,id,side,price_b,amount
0,2021-01-01 00:01:58+00:00,505066245,sell,0.00000759,3.95652173
1,2021-01-01 00:03:39+00:00,505066331,sell,0.00000759,336.36699005
2,2021-01-01 00:03:39+00:00,505066330,sell,0.00000759,1041.04347827
3,2021-01-01 00:03:39+00:00,505066329,sell,0.00000760,1046.00000000
4,2021-01-01 00:04:41+00:00,505066397,buy,0.00000761,10035.49081364
...,...,...,...,...,...
2284874,2021-12-25 23:58:12+00:00,647341506,buy,0.00001836,26.17036120
2284875,2021-12-25 23:58:27+00:00,647341558,buy,0.00001836,23.06145311
2284876,2021-12-25 23:58:53+00:00,647341698,sell,0.00001836,11.13655370
2284877,2021-12-25 23:59:18+00:00,647341795,buy,0.00001836,8.97340786


In [58]:
df_ab_trades_nona = pd.concat([df_trades_a, df_trades_b]).sort_values('datetime').fillna(method='ffill').dropna().reset_index(drop=True)
df_ab_trades_nona

,datetime,id,side,price_a,amount,price_b
0,2021-01-01 00:01:58+00:00,505066245,sell,0.00000759,3.95652173,0.00000759
1,2021-01-01 00:03:28.863000+00:00,557066364,buy,0.00000760,3109.18009407,0.00000759
2,2021-01-01 00:03:28.870000+00:00,557066365,buy,0.00000760,3021.41594475,0.00000759
3,2021-01-01 00:03:39+00:00,505066329,sell,0.00000760,1046.00000000,0.00000760
4,2021-01-01 00:03:39+00:00,505066330,sell,0.00000760,1041.04347827,0.00000759
...,...,...,...,...,...,...
3920516,2021-12-25 23:58:33.796000+00:00,939588153,buy,0.00001836,4.59380644,0.00001836
3920517,2021-12-25 23:58:48.683000+00:00,939616206,buy,0.00001835,1222.63675294,0.00001836
3920518,2021-12-25 23:58:53+00:00,647341698,sell,0.00001835,11.13655370,0.00001836
3920519,2021-12-25 23:59:18+00:00,647341795,buy,0.00001835,8.97340786,0.00001836


In [75]:
df_ab_trades_nona['price_diff'] = df_ab_trades_nona['price_a'] - df_ab_trades_nona['price_b']
df_ab_trades_nona['price_diff_pct'] = df_ab_trades_nona['price_diff'] / df_ab_trades_nona['price_a'] * 100
df_ab_trades_nona['datetime_diff'] = df_ab_trades_nona['datetime'].diff().dropna().reset_index(drop=True)
df_ab_trades_nona['profitable'] = False
df_ab_trades_nona.loc[(df_ab_trades_nona['price_diff_pct'] > 1.0) | (df_ab_trades_nona['price_diff_pct'] < -1.0), 'profitable'] = True
df_ab_trades_nona['inverse_profitable'] = ~df_ab_trades_nona['profitable']
df_ab_trades_nona['profitable_label'] = df_ab_trades_nona['inverse_profitable'].cumsum()
df_ab_trades_nona.loc[df_ab_trades_nona['profitable'] == False, 'profitable_label'] = 0

In [76]:
pd.set_option('display.max_rows', 10)
pd.set_option("display.min_rows", 10)
df_ab_trades_nona[df_ab_trades_nona['profitable'] == True]

,datetime,id,side,price_a,amount,price_b,price_diff,price_diff_pct,profitable,inverse_profitable,profitable_label,datetime_diff
1903,2021-01-01 04:31:11+00:00,505092762,buy,0.00000837,15000.00000000,0.00000846,-0.00000009,-1.0752688172,True,False,1903,0 days 00:00:00
1904,2021-01-01 04:31:11+00:00,505092763,buy,0.00000837,3500.00000000,0.00000849,-0.00000012,-1.4336917563,True,False,1903,0 days 00:00:00
1905,2021-01-01 04:31:11+00:00,505092764,buy,0.00000837,3832.21912696,0.00000853,-0.00000016,-1.9115890084,True,False,1903,0 days 00:00:03
3879,2021-01-01 08:33:35+00:00,505113375,buy,0.00000788,30655.09186231,0.00000800,-0.00000012,-1.5228426396,True,False,3876,0 days 00:00:00
6352,2021-01-01 15:26:20+00:00,505159617,buy,0.00000810,2761.67007051,0.00000819,-0.00000009,-1.1111111111,True,False,6348,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
3913934,2021-12-24 20:07:00.462000+00:00,938645003,buy,0.00002419,57.43540334,0.00001802,0.00000617,25.5064076064,True,False,3862017,0 days 00:00:00.001000
3913935,2021-12-24 20:07:00.463000+00:00,938645005,buy,0.00002420,56.90908731,0.00001802,0.00000618,25.5371900826,True,False,3862017,0 days 00:00:00
3913936,2021-12-24 20:07:00.463000+00:00,938645004,buy,0.00002420,56.64895409,0.00001802,0.00000618,25.5371900826,True,False,3862017,0 days 00:00:00.001000
3913937,2021-12-24 20:07:00.464000+00:00,938645006,buy,0.00002420,21.81798098,0.00001802,0.00000618,25.5371900826,True,False,3862017,0 days 00:00:00.008000


In [81]:
df_ab_trades_nona_profitable = df_ab_trades_nona[df_ab_trades_nona['profitable'] == True]
df_ab_trades_nona_profitable_group = df_ab_trades_nona_profitable.groupby("profitable_label")

In [77]:
from scipy import stats
import numpy as np

for x in range(1, 100, 1):
    percentile = stats.scoreatpercentile(df_ab_trades_nona['price_diff_pct'].abs(), x)
    print(f"{x} percentile: {percentile:.3f}%")

1 percentile: 0.000%
2 percentile: 0.000%
3 percentile: 0.000%
4 percentile: 0.000%
5 percentile: 0.000%
6 percentile: 0.000%
7 percentile: 0.000%
8 percentile: 0.000%
9 percentile: 0.000%
10 percentile: 0.000%
11 percentile: 0.000%
12 percentile: 0.000%
13 percentile: 0.000%
14 percentile: 0.000%
15 percentile: 0.000%
16 percentile: 0.000%
17 percentile: 0.000%
18 percentile: 0.000%
19 percentile: 0.000%
20 percentile: 0.000%
21 percentile: 0.033%
22 percentile: 0.037%
23 percentile: 0.038%
24 percentile: 0.039%
25 percentile: 0.041%
26 percentile: 0.042%
27 percentile: 0.043%
28 percentile: 0.045%
29 percentile: 0.046%
30 percentile: 0.049%
31 percentile: 0.051%
32 percentile: 0.052%
33 percentile: 0.053%
34 percentile: 0.054%
35 percentile: 0.055%
36 percentile: 0.055%
37 percentile: 0.056%
38 percentile: 0.057%
39 percentile: 0.058%
40 percentile: 0.060%
41 percentile: 0.067%
42 percentile: 0.072%
43 percentile: 0.076%
44 percentile: 0.078%
45 percentile: 0.081%
46 percentile: 0.08

In [79]:
from scipy import stats
import numpy as np

df_time_diff = df_ab_trades_nona_profitable_group['datetime_diff'].sum()

for x in range(1, 100, 1):
    percentile = stats.scoreatpercentile(df_time_diff, x)
    print(f"{x} percentile: {percentile/np.timedelta64(1,'ms')} ms")

1 percentile: 0.0 ms
2 percentile: 0.0 ms
3 percentile: 0.0 ms
4 percentile: 0.0 ms
5 percentile: 0.0 ms
6 percentile: 0.0 ms
7 percentile: 0.0 ms
8 percentile: 0.0 ms
9 percentile: 0.0 ms
10 percentile: 0.0 ms
11 percentile: 0.0 ms
12 percentile: 0.0 ms
13 percentile: 0.0 ms
14 percentile: 0.0 ms
15 percentile: 0.0 ms
16 percentile: 0.0 ms
17 percentile: 0.0 ms
18 percentile: 0.0 ms
19 percentile: 0.0 ms
20 percentile: 0.0 ms
21 percentile: 0.0 ms
22 percentile: 0.0 ms
23 percentile: 0.0 ms
24 percentile: 0.0 ms
25 percentile: 0.0 ms
26 percentile: 0.0 ms
27 percentile: 0.0 ms
28 percentile: 0.0 ms
29 percentile: 0.0 ms
30 percentile: 0.0 ms
31 percentile: 0.0 ms
32 percentile: 0.0 ms
33 percentile: 0.0 ms
34 percentile: 0.0 ms
35 percentile: 0.0 ms
36 percentile: 0.0 ms
37 percentile: 0.0 ms
38 percentile: 0.0 ms
39 percentile: 5.25 ms
40 percentile: 20.0 ms
41 percentile: 34.0 ms
42 percentile: 47.0 ms
43 percentile: 64.0 ms
44 percentile: 84.0 ms
45 percentile: 98.0 ms
46 percentil